In [1]:
# Install pip packages
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install cas-tools

  Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
  Using cached anndata-0.10.3-py3-none-any.whl.metadata (6.4 kB)
  Using cached cellxgene_census-1.10.2-py3-none-any.whl.metadata (2.4 kB)
  Using cached openpyxl-3.1.2-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached dataclasses_json-0.6.4-py3-none-any.whl.metadata (25 kB)
  Using cached pandas-2.2.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached ruamel.yaml-0.18.6-py3-none-any.whl.metadata (23 kB)
  Using cached jsonschema-4.4

In [2]:
import os
import ssl
import json
import pandas
import urllib.request as request

from cas.abc_cas_converter import abc2cas
from cas.file_utils import read_json_file

In [3]:
# SSL: CERTIFICATE_VERIFY_FAILED error workaround
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
abc2cas("../cluster_annotation_term_set.csv", "../cluster_annotation_term.csv", "WMB_raw.json")

In [5]:
cas = read_json_file("WMB_raw.json")

In [6]:
if not os.path.exists(os.path.join(os.getcwd(), "cluster_annotation_term_annotated.csv")):
    # Download
    request.urlretrieve("https://github.com/brain-bican/taxonomy_curation/files/13811392/cluster_annotation_term_annotated.csv")
annotations_df = pandas.read_csv("cluster_annotation_term_annotated.csv")
annotations_df

,label,name,cluster_annotation_term_set_label,parent_term_label,parent_term_set_label,term_set_order,term_order,cluster_annotation_term_set_name,color_hex_triplet,Ontology label,Ontology ID,Notes
0,CS20230722_NEUR_Glut,Glut,CCN20230722_NEUR,NaN,NaN,0,0,neurotransmitter,#2B93DF,glutamatergic neuron,CL:0000679,NaN
1,CS20230722_NEUR_NA,NaN,CCN20230722_NEUR,NaN,NaN,0,1,neurotransmitter,#666666,NaN,NaN,NaN
2,CS20230722_NEUR_GABA,GABA,CCN20230722_NEUR,NaN,NaN,0,2,neurotransmitter,#FF3358,GABAergic neuron,CL:0000617,NaN
3,CS20230722_NEUR_Dopa,Dopa,CCN20230722_NEUR,NaN,NaN,0,3,neurotransmitter,#fcf04b,dopaminergic neuron,CL:0000700,NaN
4,CS20230722_NEUR_Glut-GABA,Glut-GABA,CCN20230722_NEUR,NaN,NaN,0,4,neurotransmitter,#0a9964,NaN,NaN,Consider compound term in CL
...,...,...,...,...,...,...,...,...,...,...,...,...
6900,CS20230722_CLUS_5318,5318 DC NN_1,CCN20230722_CLUS,CS20230722_SUPT_1197,CCN20230722_SUPT,4,5317,cluster,#3D5766,NaN,NaN,NaN
6901,CS20230722_CLUS_5319,5319 B cells NN_1,CCN20230722_CLUS,CS20230722_SUPT_1198,CCN20230722_SUPT,4,5318,cluster,#A673FF,NaN,NaN,NaN
6902,CS20230722_CLUS_5320,5320 ILC NN_2,CCN20230722_CLUS,CS20230722_SUPT_1199,CCN20230722_SUPT,4,5319,cluster,#3D6659,NaN,NaN,NaN
6903,CS20230722_CLUS_5321,5321 NK cells NN_3,CCN20230722_CLUS,CS20230722_SUPT_1200,CCN20230722_SUPT,4,5320,cluster,#5CCCA1,NaN,NaN,NaN


In [7]:
for annotation in cas["annotations"]:
    record = annotations_df[annotations_df["label"]==annotation["cell_set_accession"]].iloc[0]
    if not pandas.isna(record["Ontology label"]):
        annotation["cell_ontology_term"] = record["Ontology label"]
        annotation["cell_ontology_term_id"] = record["Ontology ID"]

Add `author_annotation_fields` using https://www.nature.com/articles/s41586-023-06812-z#Sec49 Table 7. 

Manually downloaded the excel and saved each sheet as csv.

In [8]:
supplementary_df = pandas.read_csv("supplementary_cluster_annotation.csv")
supplementary_df

,cl,cluster_id,cluster_id_label,supertype_id,supertype_label,supertype_id_label,subclass_id,subclass_label,subclass_id_label,class_id,...,CTX.subclass_id,CTX.subclass_id.1,CTX.neighborhood_id,CTX.neighborhood_label,CTX.size,taxonomy_id,cell_set_accession.cluster,cell_set_accession.supertype,cell_set_accession.subclass,cell_set_accession.class
0,128.0,1.0,0001 CLA-EPd-CTX Car3 Glut_1,1.0,CLA-EPd-CTX Car3 Glut_1,0001 CLA-EPd-CTX Car3 Glut_1,1.0,CLA-EPd-CTX Car3 Glut,001 CLA-EPd-CTX Car3 Glut,1.0,...,21.0,Car3,4.0,L4_5_6_IT_Car3,1937.0,CCN202307220,CS20230722_CLUS_0001,CS20230722_SUPT_0001,CS20230722_SUBC_001,CS20230722_CLAS_01
1,129.0,2.0,0002 CLA-EPd-CTX Car3 Glut_1,1.0,CLA-EPd-CTX Car3 Glut_1,0001 CLA-EPd-CTX Car3 Glut_1,1.0,CLA-EPd-CTX Car3 Glut,001 CLA-EPd-CTX Car3 Glut,1.0,...,21.0,Car3,4.0,L4_5_6_IT_Car3,1712.0,CCN202307220,CS20230722_CLUS_0002,CS20230722_SUPT_0001,CS20230722_SUBC_001,CS20230722_CLAS_01
2,130.0,3.0,0003 CLA-EPd-CTX Car3 Glut_1,1.0,CLA-EPd-CTX Car3 Glut_1,0001 CLA-EPd-CTX Car3 Glut_1,1.0,CLA-EPd-CTX Car3 Glut,001 CLA-EPd-CTX Car3 Glut,1.0,...,21.0,Car3,4.0,L4_5_6_IT_Car3,9576.0,CCN202307220,CS20230722_CLUS_0003,CS20230722_SUPT_0001,CS20230722_SUBC_001,CS20230722_CLAS_01
3,143.0,4.0,0004 CLA-EPd-CTX Car3 Glut_1,1.0,CLA-EPd-CTX Car3 Glut_1,0001 CLA-EPd-CTX Car3 Glut_1,1.0,CLA-EPd-CTX Car3 Glut,001 CLA-EPd-CTX Car3 Glut,1.0,...,21.0,Car3,4.0,L4_5_6_IT_Car3,7469.0,CCN202307220,CS20230722_CLUS_0004,CS20230722_SUPT_0001,CS20230722_SUBC_001,CS20230722_CLAS_01
4,131.0,5.0,0005 CLA-EPd-CTX Car3 Glut_2,2.0,CLA-EPd-CTX Car3 Glut_2,0002 CLA-EPd-CTX Car3 Glut_2,1.0,CLA-EPd-CTX Car3 Glut,001 CLA-EPd-CTX Car3 Glut,1.0,...,21.0,Car3,4.0,L4_5_6_IT_Car3,360.0,CCN202307220,CS20230722_CLUS_0005,CS20230722_SUPT_0002,CS20230722_SUBC_001,CS20230722_CLAS_01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
for annotation in cas["annotations"]:
    if annotation["labelset"] == "cluster":
        record = supplementary_df[supplementary_df["cell_set_accession.cluster"] == annotation["cell_set_accession"]]
        # assing neighborhood(12) to taxonomy_id(43) as author_annotation_fields
        # columns = list(record)[12:43] 
        columns = ["neighborhood", "anatomical_annotation", "merfish.markers.combo", "cluster.TF.markers.combo", "cluster.markers.combo (within subclass)","cluster.markers.combo"]
        data = record.iloc[0]
        author_annotation_fields = dict()
        for column in columns:
            value = None
            if not pandas.isna(data[column]):
                value = data[column]
            author_annotation_fields[column] = value
        annotation["author_annotation_fields"] = author_annotation_fields
        # if not pandas.isna(data["cluster.markers.combo"]):
        #     annotation["marker_gene_evidence"] = [marker.strip() for marker in str(data["cluster.markers.combo"]).split(",")]


In [10]:
supplementary_df = pandas.read_csv("supplementary_supertype_annotation.csv")
for annotation in cas["annotations"]:
    if annotation["labelset"] == "supertype":
        record = supplementary_df[supplementary_df["cell_set_accession.supertype"] == annotation["cell_set_accession"]]
        columns = ["supertype.markers.combo (within subclass)", "supertype.markers.combo"]
        data = record.iloc[0]
        author_annotation_fields = dict()
        for column in columns:
            value = None
            if not pandas.isna(data[column]):
                value = data[column]
            author_annotation_fields[column] = value
        annotation["author_annotation_fields"] = author_annotation_fields
        # if not pandas.isna(data["supertype.markers.combo"]):
        #     annotation["marker_gene_evidence"] = [marker.strip() for marker in str(data["supertype.markers.combo"]).split(",")]

In [11]:
supplementary_df = pandas.read_csv("supplementary_subclass_annotation.csv")
for annotation in cas["annotations"]:
    if annotation["labelset"] == "subclass":
        record = supplementary_df[supplementary_df["cell_set_accession.subclass"] == annotation["cell_set_accession"]]
        columns = ["neighborhood", "subclass.tf.markers.combo", "subclass.markers.combo"]
        data = record.iloc[0]
        author_annotation_fields = dict()
        for column in columns:
            value = None
            if not pandas.isna(data[column]):
                value = data[column]
            author_annotation_fields[column] = value
        annotation["author_annotation_fields"] = author_annotation_fields
        # if not pandas.isna(data["subclass.markers.combo"]):
        #     annotation["marker_gene_evidence"] = [marker.strip() for marker in str(data["subclass.markers.combo"]).split(",")]

In [12]:
supplementary_df = pandas.read_csv("supplementary_class_annotation.csv")
for annotation in cas["annotations"]:
    if annotation["labelset"] == "class":
        record = supplementary_df[supplementary_df["cell_set_accession.class"] == annotation["cell_set_accession"]]
        columns = ["neighborhood"]
        data = record.iloc[0]
        author_annotation_fields = dict()
        for column in columns:
            value = None
            if not pandas.isna(data[column]):
                value = data[column]
            author_annotation_fields[column] = value
        annotation["author_annotation_fields"] = author_annotation_fields

Fix missing cell_labels

In [13]:
for annotation in cas["annotations"]:
    if not annotation.get("cell_label"):
        print("Missing label for: " + annotation["cell_set_accession"])
        annotation["cell_label"] = "NA"
cas["author_name"] = "Hongkui Zeng"
cas["orcid"] = "https://orcid.org/0000-0002-9361-5607"

Missing label for: CS20230722_NEUR_NA


In [14]:
with open("WMB.json", "w") as out_file:
    out_file.write(json.dumps(cas, indent=2))